In [1]:
import os

from os.path import  join, isdir
from plan import Plan
from action import Action
from utils import load_from_folder
from multiprocess import Pool
import random
from logging import exception
import re
import shutil

random.seed(42)

In [70]:
save_dir = './new_plans/'
data_base_dir = '../datasets/'
domain = 'logistics'
results_dir = f"{save_dir}/{domain}/"   
source_dir = f"{join(data_base_dir, domain)}/optimal_plans/dictionaries_and_plans/" 
print('Domain dir:', source_dir)
os.makedirs(save_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

plans_to_process = 1000 # number of plans to process
versions_per_plan = 6 # number of versions per each plan
number_of_goals = 4 # number of goals per each new plan
test = False # test will process only 3 plans
rec_classes = [[0,0.1], [0.1,0.2], [0.2,0.3], [0.3,0.4], [0.4,0.5], [0.5,0.6], [0.6,0.7], [0.7,0.8], [0.8,0.9], [0.9,1]] # classes of recognizability

Domain dir: ../datasets/logistics/optimal_plans/dictionaries_and_plans/


In [67]:
#to clear the results directory
for folder in os.listdir(results_dir):
    #remove directory if it exists along with its content
    if isdir(join(results_dir, folder)):
        shutil.rmtree(join(results_dir, folder))

In [ ]:
#loading the plans
plans = load_from_folder(source_dir,["plans"])[0]
print(f"Plans: {len(plans)}")

plans loaded from ../datasets/logistics/optimal_plans/dictionaries_and_plans/
Plans: 47769


In [59]:
def compute_recognizability(current_goal_state, goal_state_list):
    """
    Compute the difficulty of a plan.
    :param current_goal_state: The goal state for which we calculate the recognizability.
    :param goal_state_list: The list of goal states to use for the computation.
    :return: The recognizability of the plan.
    """
    
    #the current goal state must be in the goal state list
    if current_goal_state not in goal_state_list:
        raise ValueError(f"current_goal_state {current_goal_state} must be included in goal_state_list {goal_state_list}")
    
    #min and max to use for normalization
    #min recognizability is when all the fluent in current are present in each goal state in the goal state list
    #the formula becomes 1+1+1+...+1 = len(goal_state_list)
    #max recognizability is when all the fluent in current are not present anywhere in the goal state list
    rec_when_least_recognisable  = 1*len(current_goal_state)
    rec_when_most_recognisable  = 1/(len(goal_state_list)) * len(current_goal_state)
    
    # print(f"rec_when_least_recognisable: {rec_when_least_recognisable}")
    # print(f"rec_when_most_recognisable: {rec_when_most_recognisable}")
    
    sum = 0
    #need to count how many times the current goal fluent is in the goal state list
    for current_goal_fluent in current_goal_state:
        count = 0
        for goal_state in goal_state_list:
            for goal_fluent in goal_state:       
                if current_goal_fluent==goal_fluent:
                    count += 1
                    break
        sum += 1/count
    
    #print(f"Unscaled recognizability: {sum}")
    
    #normalize the recognizability 
    recognizability = (sum-rec_when_most_recognisable) / (rec_when_least_recognisable-rec_when_most_recognisable)
    
    return round(recognizability, 4)
                    

In [56]:
#test compute_recognizability
goal_state_list = [["a", "b", "c"], 
                   ["a", "e", "f"], 
                   ["g", "h", "i"], ]
print(compute_recognizability(goal_state_list[0], goal_state_list))

rec_when_least_recognisable: 3
rec_when_most_recognisable: 1.0
0.75


In [7]:
#compute difficulty testing
# current_goal_state = [6, 5, 7, 9]
# goal_state_list = [[1, 2, 3, 4], 
#                    [1, 2, 3, 4], 
#                    [1, 2, 3, 4], 
#                    [1, 2, 3, 4], 
#                    [1, 2, 3, 4]]
# goal_state_list.append(current_goal_state)
# rec = compute_recognizability(current_goal_state, goal_state_list)
# print(f"recognizability : {rec}")

In [8]:
    #?tried to have a way to generate the goal state list directly with a given recognizability
    #?kinda works, but approximation is too large, and it is unncessarily complex
    
    #todo check code again, gpt helped with it
    
# import math

# def approx_counts(num_of_fluents: int, num_of_states: int, r: float) -> list:
#     """
#     Return a list of n integer counts in [1..m] whose reciprocals sum
#     approximately to S = n/m + r * n*(m-1)/m.

#     Strategy:
#       1. Compute the “continuous” count c = m / (1 + r*(m-1)).
#       2. If c is (nearly) integer, just return [round(c)]*n.
#       3. Otherwise let b = floor(c), B = b + 1, and solve for x:
#             x*(1/b) + (n-x)*(1/B) = S
#          => x = (n/B - S) / (1/B - 1/b)
#       4. Round x to the nearest integer; assign x entries = b, (n-x) = B.
#       5. Clip to [0..n] and, if desired, do a tiny local tweak
#          (e.g. move 1 count up/down) to reduce the residual error.
#     """
#     # 1) target unscaled sum
#     S = (num_of_fluents/num_of_states) + r * (num_of_fluents * (num_of_states-1) / num_of_states)

#     # 2) continuous ideal count
#     c = num_of_states / (1 + r*(num_of_states-1))
#     c_round = round(c)
#     # if it’s essentially integral, use it
#     if abs(c_round - c) < 1e-6 or c_round in (1, num_of_states):
#         return [c_round] * num_of_fluents

#     # 3) floor / ceil
#     b = math.floor(c)
#     B = b + 1

#     # solve x*(1/b) + (n-x)*(1/B) = S
#     #   => x = (n/B - S) / (1/B - 1/b)
#     denom = (1/B - 1/b)
#     if abs(denom) < 1e-8:
#         # degenerate; fallback to uniform
#         return [c_round] * num_of_fluents

#     x_real = (num_of_fluents/B - S) / denom
#     x = int(round(x_real))

#     # 4) clip and build
#     x = max(0, min(num_of_fluents, x))
#     counts = [b]*x + [B]*(num_of_fluents - x)

#     # 5) (optional) tiny local corrections
#     # compute residual error
#     current_sum = sum(1/ci for ci in counts)
#     # if we’re off by more than, say, 1/n, try one adjustment
#     if abs(current_sum - S) > 1e-3:
#         # if sum too small, we need to increase it ⇒ lower some ci by 1
#         if current_sum < S:
#             # find an index with ci > 1 and decrement it
#             for i in range(num_of_fluents):
#                 if counts[i] > 1:
#                     counts[i] -= 1
#                     break
#         else:
#             # sum too big ⇒ decrement sum ⇒ increase some ci by 1
#             for i in range(num_of_fluents):
#                 if counts[i] < num_of_states:
#                     counts[i] += 1
#                     break

#     return counts



# def build_goal_list(current, goal_set_list_size, counts, fillers=['a','b','c','d','e']):
#     num_of_goal_fluents = len(current)
#     # initialize empty slots
#     slots = [ [] for _ in range(goal_set_list_size) ]
#     # for each fluent, choose which rows it goes in
#     for fluent, c in zip(current, counts):
#         rows = random.sample(range(goal_set_list_size), c)
#         for r in rows:
#             slots[r].append(fluent)
#     # fill the rest with distractors
#     all_distractors = ['a','b','c','d','e']  # pool of non-current fluents
#     for r in range(goal_set_list_size):
#         while len(slots[r]) < num_of_goal_fluents:
#             slots[r].append(random.choice(all_distractors))
#         random.shuffle(slots[r])
#     return slots

# # Example usage:
# current = [6,5,7,9]
# goal_set_list_size = 6
# # counts chosen by solving sum(1/ci)=S (approximated)
# counts = approx_counts(len(current), goal_set_list_size, 0.47)
# print("Counts:", counts)
# goal_list = build_goal_list(current, goal_set_list_size, counts)
# print("Goal list:")
# for i, g in enumerate(goal_list):
#     print(f"Goal {i}: {g}")
# rec = compute_recognizability(current, goal_list)
# print(f"recognizability : {rec}")

In [ ]:
 #todo subject to change depending on how we want output
def write_and_save_versions(plan, goal_state_list, obj_set_dict={}, rec_class=[0,1]):
    """Write the plan and its versions to files.
    :param plan: The plan to write.
    :param goal_state_list: The list of goal states, each state will produce a different version, at index 0 should be the original plan.
    :param obj_set_dict: The dictionary of objects.
    :return: None
    """
        
    for i, goal_state in enumerate(goal_state_list):
        
        #extract plan name with regex
        name = re.search(r"(p\d+)(?=\.)", plan.plan_name).group(1)
        
        #definition
        new_problem = ""
        # if i==0:
        #     new_problem += f";;(;metadata (recognizability:{round(compute_recognizability(goal_state_list[0], goal_state_list),2)})\n"
        
        new_problem += f"(define (problem {domain}_{name}_{i})\n(:domain {domain})\n(:objects\n\t"
    
        
        #objects in a dict format, {type: obj_set}
        for type, obj_set in obj_set_dict.items():
            if len(obj_set) > 0:
                for obj in obj_set:
                    new_problem += f"{obj} "
                new_problem += f"- {type}\n\t"
        new_problem += f")\n"
        
        #initial state
        new_problem += f"(:init\n"
        for fluent in plan.initial_state:
            new_problem += f"\t{fluent}\n"
        new_problem += f")\n"
        
        #goal state
        new_problem += f"(:goal (and\n"
        for goal in goal_state:
            new_problem += f"\t{goal}\n"
        new_problem += f"))\n)"
        #print(new_problem + "\n\n")
        
        #save the new problem in a file
        #naming convention is {current plan name_version number.pddl}, _0 is the original plan
        new_problem_dir = f"{results_dir}/{name}/"
        class_dir = f"{results_dir}/{name}/{rec_class[0]}_{rec_class[1]}/"
        os.makedirs(new_problem_dir, exist_ok=True)
        os.makedirs(class_dir, exist_ok=True)
        new_problem_file = f"{class_dir}/{name}_{i}.pddl"
        with open(new_problem_file, "w") as f:
            f.write(new_problem)

In [ ]:
 #todo subject to change depending on how we want output

def write_original_plan(plan, goal_state, obj_set_dict):
    
    #extract plan name with regex
    name = re.search(r"(p\d+)(?=\.)", plan.plan_name).group(1)
    
    #definition
    new_problem = ""
    
    new_problem += f"(define (problem {domain}_{name}_og)\n(:domain {domain})\n(:objects\n\t"

    
    #objects in a dict format, {type: obj_set}
    for type, obj_set in obj_set_dict.items():
        if len(obj_set) > 0:
            for obj in obj_set:
                new_problem += f"{obj} "
            new_problem += f"- {type}\n\t"
    new_problem += f")\n"
    
    #initial state
    new_problem += f"(:init\n"
    for fluent in plan.initial_state:
        new_problem += f"\t{fluent}\n"
    new_problem += f")\n"
    
    #goal state
    new_problem += f"(:goal (and\n"
    for goal in goal_state:
        new_problem += f"\t{goal}\n"
    new_problem += f"))\n)"
    #print(new_problem + "\n\n")
    
    #save the new problem in a file
    #naming convention is {current plan name_version number.pddl}
    new_problem_dir = f"{results_dir}/{name}/"
    os.makedirs(new_problem_dir, exist_ok=True)
    new_problem_file = f"{new_problem_dir}/{name}_og.pddl"
    with open(new_problem_file, "w") as f:
        f.write(new_problem)

In [ ]:
def create_goal_state_list(package_for_goal_set, pos_for_goal_set, number_of_goals, versions_per_plan):
    """Create a list of goal states.
    :param package_for_goal_set: The set of packages to use for the goal state.
    :param pos_for_goal_set: The set of positions to use for the goal state.
    :param number_of_goals: The number of goals to generate.
    :param versions_per_plan: The number of versions to generate.
    :return: A list of goal states.
    """	
    goal_state_list = []
    for i in range(0, versions_per_plan + 1):
        # generate a random goal state
        goal_state = generate_goal_state(package_for_goal_set, pos_for_goal_set, number_of_goals)
            
        goal_state_list.append(goal_state)
        
    for i, goal_state in enumerate(goal_state_list):
        #convert the sets to a list
        goal_state_list[i] = list(goal_state)
        
    return goal_state_list
        
def generate_goal_state(package_for_goal_set, pos_for_goal_set, number_of_goals):
    """Generate a random goal state.	
    :param package_for_goal_set: The set of packages to use for the goal state.
    :param pos_for_goal_set: The set of positions to use for the goal state.
    :param number_of_goals: The number of fluents to generate for the goal state.
    :return: A set of fluents representing a goal state.
    """
    goal_state = set()
    package_for_goal_set_copy = package_for_goal_set.copy()
    pos_for_goal_set_copy = pos_for_goal_set.copy()
    for _ in range(number_of_goals):
        random_package = random.choice(list(package_for_goal_set_copy))
        package_for_goal_set_copy.remove(random_package)
        random_pos = random.choice(list(pos_for_goal_set_copy))
        #pos_for_goal_set_copy.remove(random_pos)
        goal_state.add(f"at {random_package} {random_pos}") 
    return goal_state 

In [ ]:
#to calcculate how precise the generation is
global_counter = 0

running_sum_rec_error = 0

In [ ]:
def check_consistency(goal_state, prefix="obj"):
    """	
    Check if the goal state is consistent by checking if it has same obj used more than once.
    :param goal_state: The goal state to check.
    :return: True if the goal state is consistent, False otherwise.
    """
    objects = []
    for fluent in goal_state:
        obj = re.search(rf"{prefix}\d+", fluent)
        objects.append(obj.group(0))
    return len(objects) == len(set(objects))
        

def check_if_fluent_is_usable(fluent_to_add, goal_state):
    """
    Check if the fluent to add is usable in the goal state.
    This means checking if fluent is not already in the goal state and if the object in fluent is not already in the goal state.
    :param fluent_to_add: The fluent to add.
    :param goal_state: The goal state to check.
    :return: True if the fluent to add is usable, False otherwise.
    """
    #check if goal state is a list of strings
    if isinstance(fluent_to_add, list):
        raise ValueError(f"fluent to add is a list: {fluent_to_add}")
    for fluent in goal_state:
        if fluent == fluent_to_add:
            return False
        elif check_same_object_in_fluents(fluent_to_add, fluent, prefix="obj"):
            return False
    return True

#todo fix to introduce prefix
def check_same_object_in_fluents(fluent1, fluent2, prefix="obj"):
    """
    Check if the object in fluent1 is in fluent2.
    :return: True if the object in fluent1 is in fluent2, False otherwise.
    """
    #extract the object from the fluent using regex
    obj1 = re.search(rf"{prefix}\d+", fluent1).group(0)
    obj2 = re.search(rf"{prefix}\d+", fluent2).group(0)

    if obj1 == obj2:
        return True
    return False

def adapt_goal_state_list_to_recognizability(base_goal_state, goal_state_list, 
                                             package_for_goal_set, pos_for_goal_set, 
                                             number_of_goals, rec_target=[0.1, 0.2], 
                                             randomness_patience=5, regeneration_patience=3):
    """
    Adapt the goal state list to the recognizability. 
    This is done by swapping fluents between the base goal state and the goal state list.
    If we have to reduce recognisability, we swap a random fluent in the goal state list with a fluent from the base goal state.
    If we have to increase recognisability, we swap a fluent that is in the base goal state and in also in a goal in goal state list with a random possible fluent.
    This does not guarantee that the recognizability will be in the target range, but it will be close.
    It can happen that a goal state will be stuck in a local minimum, so we regenerate it to try staring from another point.
    We keep track of which regenerations we have done and if we reach the patience limit, we will use the one that is closest to the target recognizability.
    :param base_goal_state: The base goal state.
    :param goal_state_list: The list of goal states to use to compute recognizability, without base goal state.
    :param randomness_patience: The number of times we can try to adapt the goal state list before regenerating it. This will usually exhaust if we have many states that are stuck in a local minimum. If we reach this limit, we will regenerate the next stuck goal state.
    :param regeneration_patience: The number of times we can regenerate a goal state before giving up.
    :param package_for_goal_set: The set of packages to use for the goal state.
    :param pos_for_goal_set: The set of positions to use for the goal state.
    :param number_of_goals: The number of goals to generate.
    :param rec_target: The target range of recognizability.
    :return: The adapted goal state list.
    """
    #variables to keep track of errors
    global global_counter
    global running_sum_rec_error
    
    
    randomness_patience_constant = randomness_patience
    
    #to keep track of the goal states that we have regenerated
    goal_regeneration_dict = {}
    
    #we identify all the fluents that are in the goal state list    
    all_goal_fluents = []
    for goal_state in goal_state_list:
        for fluent in goal_state:
            if fluent not in all_goal_fluents:
                all_goal_fluents.append(fluent)
    
    #we identify all the fluents that are not in the base goal state but are in the goal state list
    non_base_goal_fluents = []
    for fluent in all_goal_fluents:
        if fluent not in base_goal_state:
            non_base_goal_fluents.append(fluent)
    
    #starting recognizability
    running_recognizability = compute_recognizability(base_goal_state, [base_goal_state] + goal_state_list)
    #print(f"Starting recognizability: {running_recognizability}, range is {recognizability}")
    
    #while the recognizability is not in the target range, we will keep adapting the goal state list
    while running_recognizability < rec_target[0] or running_recognizability > rec_target[1]:
        #print(f"Running recognizability start of step: {running_recognizability}, range is {recognizability}")
        
        #choose a random goal state from the list
        goal_state = random.choice(goal_state_list)
        
        if running_recognizability > rec_target[1]:
            #rec too high: find a goal state in goal_state_list that has a fluent that is not in the base goal state
            #swap it with one from usable_base_goal_fluents                
            
            #builds list of fluents that are in the base goal state but not in this goal state and don't introduce inconsistencies
            usable_base_goal_fluents = []
            for fluent in base_goal_state:  
                if check_if_fluent_is_usable(fluent, goal_state):
                    usable_base_goal_fluents.append(fluent)
                            
            if len(usable_base_goal_fluents) > 0:
                #swap a fluent from the goal state with one from the base goal state
                fluent_to_swap = random.choice(usable_base_goal_fluents)
                
                #builds list of fluents that are in the goal state but not in the base goal state
                candidates_list = []
                for fluent in goal_state:
                    if fluent not in base_goal_state:
                        candidates_list.append(fluent)
                
                #if there is at least one fluent to swap   
                if len(candidates_list) > 0:
                    # debug_goal_state = goal_state.copy() #debug
                    random_fluent = random.choice(candidates_list)
                    
                    # before = check_consistency(goal_state) #debug
                    #we make the swap
                    goal_state[goal_state.index(random_fluent)] = fluent_to_swap
                    # after = check_consistency(goal_state) #debug
                    # if before == True and after == False: #debug
                    #     print(f"|>|Goal state is not consistent: old goal state{debug_goal_state}\n\t base_goal_state: {base_goal_state},\n\t fluent_to_swap: {fluent_to_swap},\n\t random_fluent: {random_fluent}, \n\t usable_base_goal_fluents: {usable_base_goal_fluents}\n\n") #debug
                                    
        elif running_recognizability < rec_target[0]:
            #rec too low: choose a goal state that has a fluent from base goal and swap it with a random one
            
            #builds list of fluents that are in the base goal state and also in this goal state
            present_base_goal_fluents = []
            for fluent in base_goal_state:
                if fluent in goal_state:
                    present_base_goal_fluents.append(fluent)
            
            #if there is at least one fluent to swap                    
            if len(present_base_goal_fluents) > 0:
                
                #builds list of fluents that are in the goal state but not in the base goal state, and don't introduce inconsistencies
                candidates_list = []
                for fluent in non_base_goal_fluents:
                    if check_if_fluent_is_usable(fluent, goal_state):
                        candidates_list.append(fluent)
                
                #if there is at least one fluent to swap
                if len(candidates_list) > 0:
                    random_fluent = random.choice(candidates_list)

                    fluent_to_swap = random.choice(present_base_goal_fluents)
                    # before = check_consistency(goal_state) #debug
                    goal_state[goal_state.index(fluent_to_swap)] = random_fluent
                    # after = check_consistency(goal_state) #debug
                    # if before == True and after == False: #debug
                    #     print(f"|<| Goal state is not consistent: {goal_state}\n\t base_goal_state: {base_goal_state},\n\t fluent_to_swap: {fluent_to_swap},\n\t random_fluent: {random_fluent}\n\n") #debug
                    
        #compute the recognizability after the step
        new_recognizability = compute_recognizability(base_goal_state, [base_goal_state] + goal_state_list)
        
        #if we have not changed the recognizability, the randomness patience is reduced
        #we will try another random goal state in the next iteration 
        if new_recognizability == running_recognizability:
            randomness_patience -= 1
            
            #if we have hit too many times stuck goal states, we will regenerate the last we encountered, so the one in this iteration
            if randomness_patience == 0 and regeneration_patience > 0:
                regeneration_patience -= 1
                randomness_patience = randomness_patience_constant
                                
                # Patience reached: regenerate stuck goal_state
                new_goal_state = generate_goal_state(package_for_goal_set, pos_for_goal_set, number_of_goals)
                
                # Store the regeneration (the key is the recognizability of the prevoius configuration)
                goal_regeneration_dict[running_recognizability] = goal_state_list
                
                # Replace the goal state in the list with the new one
                goal_state = new_goal_state
            
            #if we have exhausted the regeneration patience, we will stop trying and take the best configuration we have
            elif randomness_patience == 0 and regeneration_patience == 0:
                # Patience exhausted; if we have any regenerations, choose the one with recognizability closest to target.
                if goal_regeneration_dict:
                    # Find the closest recognizability to the target
                    # must use midpoint as we could heve rec_scores both above and below the target
                    target_recognizability = (rec_target[0] + rec_target[1]) / 2 
                    closest_recognizability = min(goal_regeneration_dict.keys(), key=lambda x: abs(x - target_recognizability))
                    running_recognizability = closest_recognizability
                    goal_state_list = goal_regeneration_dict[closest_recognizability]
                print(f"Patience exhausted, breaking: Best recognizability: {running_recognizability} | Target: {rec_target}") # debug

                # to keep track of errors we do the distace to the target range
                error = abs(running_recognizability - rec_target[0]) if running_recognizability < rec_target[0] else abs(running_recognizability - rec_target[1])
                running_sum_rec_error += error
                global_counter += 1
                break
        #print(f"Running recognizability at end of step: {running_recognizability}")
        running_recognizability = new_recognizability
    return base_goal_state, goal_state_list

In [ ]:
# #check_same_object_in_fluents testing
# fluent1 = "at obj1 pos1"
# fluent2 = "at obj2 pos2"

# print(check_same_object_in_fluents(fluent1, fluent2))

False


In [78]:
count = 0
for plan in plans:
    if test:
        if count >= 3:
            break
    elif count > plans_to_process:
        break
    #begin plan processing
        
    all_obj_set = set()
    package_for_goal_set = set()
    pos_for_goal_set = set()
    
    #* find all objects in the initial state and actions
    for line in plan.initial_state:
        for obj in line.split(" ")[1:]:
            all_obj_set.add(obj)
    for action in plan.actions:
        for fluent in action.positiveEffects:
            for obj in fluent.split(" ")[1:]:
                all_obj_set.add(obj)
        for fluent in action.negativeEffects:
            for obj in fluent.split(" ")[1:]:
                all_obj_set.add(obj)
        for fluent in action.precondition:
            for obj in fluent.split(" ")[1:]:
                all_obj_set.add(obj)
    
    #split the objects in their types
    pos_set = set()
    apn_set = set()
    cit_set = set()
    apt_set = set()
    tru_set = set()
    pack_set = set()
    obj_set_dict = {}
    for obj in all_obj_set:
        if obj.startswith("pos"):
            pos_set.add(obj)
            pos_for_goal_set.add(obj) #these will be used for goal creation
        elif obj.startswith("obj"):
            pack_set.add(obj)
            package_for_goal_set.add(obj) #these will be used for goal creation
        elif obj.startswith("apn"):
            apn_set.add(obj)
        elif obj.startswith("cit"):
            cit_set.add(obj)
        elif obj.startswith("tru"):
            tru_set.add(obj)
        elif obj.startswith("apt"):
            apt_set.add(obj)
    
    #useful for printing the plan
    if len(pos_set) > 0:
        obj_set_dict["location"] = pos_set
    if len(apn_set) > 0:
        obj_set_dict["airplane"] = apn_set
    if len(cit_set) > 0:
        obj_set_dict["city"] = cit_set
    if len(apt_set) > 0:
        obj_set_dict["airport"] = apt_set
    if len(tru_set) > 0:
        obj_set_dict["truck"] = tru_set
    if len(pack_set) > 0:
        obj_set_dict["package"] = pack_set
    
    # raise an exception if number of goals > number of packages or positions
    # as we won't be able to generate a goal state with the given number of goals
    if number_of_goals > len(package_for_goal_set):
        raise exception(f"Number of goals {number_of_goals} is greater than the number of objects {len(package_for_goal_set)}")

    # in case we want that objects can not be at the same position
    # if number_of_goals > len(pos_for_goal_set):
    #     raise exception(f"Number of goals {number_of_goals} is greater than the number of positions {len(pos_for_goal_set)}")

    write_original_plan(plan=plan, goal_state=plan.goals, obj_set_dict=obj_set_dict)
    
    for interval in rec_classes:    
        goal_state_list = create_goal_state_list(package_for_goal_set=package_for_goal_set, 
                                                 pos_for_goal_set=pos_for_goal_set, 
                                                 number_of_goals=number_of_goals, 
                                                 versions_per_plan=versions_per_plan)
        
        # for goal_state in goal_state_list: # debug
        #     #check if any goal state has conflicting fluents # debug
        #     for fluent1 in goal_state: # debug
        #         for fluent2 in goal_state: # debug
        #             if fluent1 != fluent2 and check_same_object_in_fluents(fluent1, fluent2): # debug
        #                 raise exception(f"Goal state {goal_state} has conflicting fluents: {fluent1} and {fluent2}") # debug
        
        #print(f"Goals state list before: {goal_state_list}") # debug
        base_goal_state, adapted_goal_state_list = adapt_goal_state_list_to_recognizability(base_goal_state=goal_state_list[0], goal_state_list=goal_state_list[1:], 
                                                                                            package_for_goal_set=package_for_goal_set, pos_for_goal_set=pos_for_goal_set, 
                                                                                            number_of_goals=number_of_goals, rec_target=interval, 
                                                                                            randomness_patience=10, regeneration_patience=5)
        #print(f"Goals state list after adapt: {[base_goal_state] + adapted_goal_state_list}") # debug

        write_and_save_versions(plan=plan, goal_state_list=adapted_goal_state_list, obj_set_dict=obj_set_dict, rec_class=interval)

    count = count + 1
    
print("Global counter:", global_counter)
print("Running sum of recognizability error:", running_sum_rec_error)
print("Average recognizability generation error:", running_sum_rec_error/global_counter if global_counter > 0 else 0)
print("Average recognizability generation error on whole dataset:", running_sum_rec_error/(plans_to_process*versions_per_plan))

global_counter = 0

running_sum_rec_error = 0

Patience exhausted, breaking: Best recognizability: 0.1201 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.1278 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.125 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.2708 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.2708 | Target: [0.1, 0.2]
Patience exhausted, breaking: Best recognizability: 0.1111 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.1104 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.2222 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.2076 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.1056 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.1007 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.2319 | Target: [0, 0.1]
Patience exhausted, breaking: Best recognizability: 0.2076 | Ta

most errors are for the lowest classes (e.g [0, 0.1]), not always a very low rec can be easily reached.
average error is about 0.04, but looking at the whole dataset its very low and not really a problem, most problems are in their classes, i think that making the lowest class [0, 0.2] is a good compromise, reducing errors to just 0.5% instances, with the same avg error, and lower when looking at whole dataset

We have to decide dataset output format, now i make a dir for every problem: this contains subdirs for every recognizability class which contain the versions generated. 